<a href="https://colab.research.google.com/github/ahmededrisahmed/fmcs39130/blob/main/new_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import re
import os
import random
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import seaborn as sns
import sklearn
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score, f1_score, roc_curve
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import keras
from keras import layers
from keras.utils import pad_sequences
from keras.models import Sequential,load_model
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM, SpatialDropout1D
import matplotlib.pyplot as plt
import tensorflow
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
random.seed(10)

nltk.download('stopwords')
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from google.colab import files
uploaded = files.upload()

Saving amazon_alexa.tsv to amazon_alexa (1).tsv


In [6]:
data_set = pd.read_csv('amazon_alexa.tsv', sep='\t')
data_set = data_set[['verified_reviews', 'feedback']]

from google.colab import data_table
from vega_datasets import data

data_table.DataTable(data_set.head(7))

,verified_reviews,feedback
0,Love my Echo!,1
1,Loved it!,1
2,"Sometimes while playing a game, you can answer...",1
3,I have had a lot of fun with this thing. My 4 ...,1
4,Music,1
5,I received the echo as a gift. I needed anothe...,1
6,"Without having a cellphone, I cannot use many ...",1


In [7]:
data_set = pd.read_csv('amazon_alexa.tsv', sep='\\t')
data_set = data_set[['verified_reviews', 'feedback']]

data_set = data_set[data_set.verified_reviews != "Neutral"]
data_set['verified_reviews'] = data_set['verified_reviews'].apply(lambda x: x.lower())
data_set['verified_reviews'] = data_set['verified_reviews'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data_set[ data_set['feedback'] == 1 ].size)
print(data_set[ data_set['feedback'] == 0 ].size)

data_set.columns = ["verified_reviews", "feedback"]
x = data_set['verified_reviews'].values
y = data_set['feedback'].values

x_train, x_test, y_train, y_test = \
train_test_split(x, y, test_size=0.1, random_state=123)

tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(x)
xtrain= tokenizer.texts_to_sequences(x_train)
xtest= tokenizer.texts_to_sequences(x_test)
vocab_size=len(tokenizer.word_index)+1

maxlen=10
xtrain=pad_sequences(xtrain,padding='post', maxlen=maxlen)
xtest=pad_sequences(xtest,padding='post', maxlen=maxlen) 

5786
514


/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [8]:
embedding_dim=50
model =Sequential()
model.add(layers.Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=maxlen))
model.add(layers.LSTM(units=50,return_sequences=True))
model.add(layers.LSTM(units=10))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(8))
model.add(layers.Dense(1, activation="sigmoid"))
model.compile(optimizer="adam", loss="binary_crossentropy",metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 50)            225750    
                                                                 
 lstm (LSTM)                 (None, 10, 50)            20200     
                                                                 
 lstm_1 (LSTM)               (None, 10)                2440      
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense (Dense)               (None, 8)                 88        
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 248,487
Trainable params: 248,487
Non-trai

In [10]:
history = model.fit(xtrain,y_train,epochs=100, batch_size= 32, verbose=2)

Epoch 1/100
89/89 - 10s - loss: 0.3722 - accuracy: 0.9160 - 10s/epoch - 111ms/step
Epoch 2/100
89/89 - 3s - loss: 0.2830 - accuracy: 0.9178 - 3s/epoch - 37ms/step
Epoch 3/100
89/89 - 4s - loss: 0.2412 - accuracy: 0.9199 - 4s/epoch - 40ms/step
Epoch 4/100
89/89 - 3s - loss: 0.2293 - accuracy: 0.9185 - 3s/epoch - 36ms/step
Epoch 5/100
89/89 - 2s - loss: 0.2298 - accuracy: 0.9178 - 2s/epoch - 24ms/step
Epoch 6/100
89/89 - 2s - loss: 0.2244 - accuracy: 0.9182 - 2s/epoch - 22ms/step
Epoch 7/100
89/89 - 2s - loss: 0.2211 - accuracy: 0.9185 - 2s/epoch - 20ms/step
Epoch 8/100
89/89 - 1s - loss: 0.2116 - accuracy: 0.9196 - 1s/epoch - 16ms/step
Epoch 9/100
89/89 - 1s - loss: 0.2025 - accuracy: 0.9203 - 1s/epoch - 15ms/step
Epoch 10/100
89/89 - 1s - loss: 0.1990 - accuracy: 0.9203 - 1s/epoch - 15ms/step
Epoch 11/100
89/89 - 1s - loss: 0.1917 - accuracy: 0.9238 - 1s/epoch - 15ms/step
Epoch 12/100
89/89 - 1s - loss: 0.1901 - accuracy: 0.9210 - 1s/epoch - 15ms/step
Epoch 13/100
89/89 - 1s - loss: 0.

In [11]:
train, accuracy = model.evaluate(xtrain, y_train, verbose=2)
print("Training Accuracy: ", accuracy)
loss, accuracy = model.evaluate(xtest, y_test, verbose=2)
print("Test Accuracy: ", accuracy)

89/89 - 1s - loss: 0.1486 - accuracy: 0.9665 - 1s/epoch - 14ms/step
Training Accuracy:  0.9664903283119202
10/10 - 0s - loss: 0.8919 - accuracy: 0.9079 - 51ms/epoch - 5ms/step
Test Accuracy:  0.9079365134239197


In [12]:
model.save('lstm_m.h5')
# load m from single file
load_old_model = load_model('lstm_m.h5')
# make predictions

In [13]:

# import modules
import tweepy

# function to display data of each tweet
def printtweetdata(n, ith_tweet):
		print()
		print(f"Tweet {n}:")
		print(f"Username:{ith_tweet[0]}")
		print(f"Description:{ith_tweet[1]}")
		print(f"Location:{ith_tweet[2]}")
		print(f"Following Count:{ith_tweet[3]}")
		print(f"Follower Count:{ith_tweet[4]}")
		print(f"Total Tweets:{ith_tweet[5]}")
		print(f"Retweet Count:{ith_tweet[6]}")
		print(f"Tweet Text:{ith_tweet[7]}")
		print(f"Hashtags Used:{ith_tweet[8]}")


# function to perform data extraction
def scrape(words, date_since, numtweet):

		# Creating DataFrame using pandas
		db = pd.DataFrame(columns=['username',
								'description',
								'location',
								'following',
								'followers',
								'totaltweets',
								'retweetcount',
								'text',
								'hashtags'])

		# We are using .Cursor() to search
		# through twitter for the required tweets.
		# The number of tweets can be
		# restricted using .items(number of tweets)
		tweets = tweepy.Cursor(api.search,
							words, lang="en",
							since_id=date_since,
							tweet_mode='extended').items(numtweet)


		# .Cursor() returns an iterable object. Each item in
		# the iterator has various attributes
		# that you can access to
		# get information about each tweet
		list_tweets = [tweet for tweet in tweets]

		# Counter to maintain Tweet Count
		i = 1

		# we will iterate over each tweet in the
		# list for extracting information about each tweet
		for tweet in list_tweets:
				username = tweet.user.screen_name
				description = tweet.user.description
				location = tweet.user.location
				following = tweet.user.friends_count
				followers = tweet.user.followers_count
				totaltweets = tweet.user.statuses_count
				retweetcount = tweet.retweet_count
				hashtags = tweet.entities['hashtags']

				# Retweets can be distinguished by
				# a retweeted_status attribute,
				# in case it is an invalid reference,
				# except block will be executed
				try:
						text = tweet.retweeted_status.full_text
				except AttributeError:
						text = tweet.full_text
				hashtext = list()
				for j in range(0, len(hashtags)):
						hashtext.append(hashtags[j]['text'])

				# Here we are appending all the
				# extracted information in the DataFrame
				ith_tweet = [username, description,
							location, following,
							followers, totaltweets,
							retweetcount, text, hashtext]
				db.loc[len(db)] = ith_tweet

				# Function call to print tweet data on screen
				printtweetdata(i, ith_tweet)
				i = i+1
		filename = 'tweets.csv'

		# we will save our database as a CSV file.
		db.to_csv(filename)

if __name__ == '__main__':

		# Enter your own credentials obtained
		# from your developer account
		consumer_key = "sDq4iViNrUs01nRtozqjlFMXC"
		consumer_secret = "AhMHrrq8qYt4aAueIKiwxjLI2SRHZ4vpI84UMXS8IEojpdCsxt"
		access_key = "1588248654548279301-RglNooFZz3LgM5BLCmtNLN0mdS1wpz"
		access_secret = "uUx5tsnNqgb3gyFqhLaUjlEAE3YCUYl4rLPQVssYkvEUJ"


		auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
		auth.set_access_token(access_key, access_secret)
		api = tweepy.API(auth)

		# Enter Hashtag and initial date
		print("Enter Twitter HashTag to search for")
		words = input()
		print("Enter Date since The Tweets are required in yyyy-mm--dd")
		date_since = input()

		# number of tweets you want to extract in one run
		numtweet = 1000
		scrape(words, date_since, numtweet)
  # amazonalexa
		print('Scraping has completed!')

Enter Twitter HashTag to search for
alexaamazon
Enter Date since The Tweets are required in yyyy-mm--dd
2019-01-01
Scraping has completed!


In [ ]:
new_data = pd.read_csv('tweets.csv')
new_data = new_data[['text']]
new_data

In [ ]:
corpus = [] 
for i in range(len(new_data)):
  s = re.sub('[^a-zA-Z]', " ", new_data['text'][i])
  s = s.lower()
  s = s.split()
  s = [word for word in s if word not in stopwords.words('english')]
  s = ' '.join(s)
  ps = PorterStemmer()
  s = ps.stem(s)
  corpus.append(s)
corpus

pre_pro=corpus
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(pre_pro)
new_x= tokenizer.texts_to_sequences(pre_pro)
vocab_size=len(tokenizer.word_index)+1
new_x=pad_sequences(new_x,padding='post', maxlen=10)

In [ ]:
new_predict = load_old_model.predict(new_x, verbose=0)

In [ ]:
s = []
for i in new_predict:
    lable = int(i)
    if lable == 1:
        s.append(1)
    else:
        s.append(0)

new_data['feedback'] = s

new_data